In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig
from peft import PeftModel, PeftConfig
import torch
import gc

# Load tokenizer
model_name = "meta-llama/Llama-3.2-3B-Instruct"  # or the base model you trained on
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
gc.collect()
torch.cuda.empty_cache()


# Load base model in 4-bit if you used quantization
config = AutoConfig.from_pretrained(model_name)
# manually set rope_scaling to supported structure:
config.rope_scaling = {"type": "dynamic", "factor": 2.0}
config.use_cache = True

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    config=config,
    torch_dtype=torch.float16
)

# # Load your LoRA adapter
# adapter_path = "./../Training/final_adapter_with_eval_1"  # or wherever your adapter_model.safetensors is
# adapted_model= PeftModel.from_pretrained(base_model, adapter_path)


2025-03-27 10:41:41.044966: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
#Need to empty_cache other wise 2 models' results will bleed into each other
# only slice and decode the new tokens 
def generate_summary(input_text, adapted_model, max_new_tokens=150):

    prompt = f"""Summarize:\n{input_text} Summary:\n"""

#     prompt = f"""Without commentary, from its original language summarize to English on useful information including sensitive data, below 100 words. If no meaning return <NULL>
# Text:
# {input_text}

    
    inputs = tokenizer(prompt, return_tensors="pt").to(adapted_model.device)
    
    with torch.no_grad():
        outputs = adapted_model.generate(
            **inputs,
            do_sample=True,
            temperature=0.7,
            max_new_tokens=max_new_tokens,
            top_p=0.9
        )
    input_len = inputs["input_ids"].shape[1]
    new_tokens = outputs[0][input_len:]  # exclude prompt
    summary = tokenizer.decode(new_tokens, skip_special_tokens=True)
    return summary

def generate_base_summary(input_text, base_model, max_new_tokens=150):
    prompt = f"""Summarize to less than 100 words:\n{input_text} Summary:\n"""
#     prompt = f"""Without commentary, from its original language summarize to English on useful information including sensitive data, below 100 words. If no meaning return <NULL>
# Text:
# {input_text}
# """
    
    inputs = tokenizer(prompt, return_tensors="pt").to(base_model.device)
    
    with torch.no_grad():
        outputs = base_model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=0.7,
            top_p=0.9
        )
    input_len = inputs["input_ids"].shape[1]
    new_tokens = outputs[0][input_len:]  # exclude prompt
    summary = tokenizer.decode(new_tokens, skip_special_tokens=True)
    return summary

def chunk_text_with_overlap(txt_file="", chunk_size=500, overlap=100, string=""):
    """Splits text into overlapping chunks."""
    if txt_file:
        with open(txt_file, 'r', encoding='utf-8') as f:
            text = f.read()
    if string:
        text = string
    chunks = []
    start = 0
    while start < len(text):
        end_ptr = min(start + chunk_size, len(text))
        chunks.append(text[start:end_ptr])
        start += chunk_size - overlap  # Move forward while keeping overlap

    # for chunk in chunks:
    #     print(chunk)
    return chunks


def rate(summary):
        prompt = f"?Is this useful:{summary}"
        inputs = tokenizer(prompt, return_tensors="pt").to(base_model.device)
    
        with torch.no_grad():
            outputs = base_model.generate(
                **inputs,
                do_sample=True,
                temperature=0.7,
                max_new_tokens=10,
                top_p=0.9
            )
        input_len = inputs["input_ids"].shape[1]
        new_tokens = outputs[0][input_len:]  # exclude prompt
        score = tokenizer.decode(new_tokens, skip_special_tokens=True)
        print("rating:", score)
        # try:
        #     score  = float(score)
        # except :
        #     score = 0
        return score 

    

In [3]:
def clean_summary(text):
    return (
        text.replace('NULL', '')
            .replace('<', '')
            .replace('>', '')
            .replace('\n', '')
            .replace('--', '')
            .replace('Summary:', '')
    )
class IterativeSummarizer:
    def __init__(self,model):
         self.model = model
     
    def summarize(self, chunks, max_token):
          if len(chunks) == 1:
               return chunks[0]  # Base case: only one chunk remains

          summarized_chunks = []

          for chunk in chunks:
               summary = generate_base_summary(chunk, base_model=self.model, max_new_tokens=max_token)
               cleaned = clean_summary(summary)
               summarized_chunks.append(cleaned)

          # Combine and re-chunk
          combined = " ".join(summarized_chunks)
          combined = combined.strip() 
          print("<----Combined--->", combined)

          # Recursively call after re-chunking
          new_chunks = chunk_text_with_overlap(string=combined, chunk_size=900)
          return self.summarize(new_chunks, max_token)


In [ ]:
import json
from pathlib import Path
evaluate_data_folder = Path("./Data/PDFs")
pdf_names = []
for txt_file in evaluate_data_folder.rglob("*.txt"):  # Get all .txt files in the folder
        tup = tuple([str(txt_file.name),txt_file])
        pdf_names.append(tup)

#print(pdf_names)
pdf_names.sort(key=lambda x: x[0])
print(len(pdf_names))
for p in pdf_names:
        print((p))
summarizer = IterativeSummarizer(model=base_model)
results = []
page = 0
page_size = 0
with open("base_inference.jsonl", "a") as f:
        for i in range(6, len(pdf_names)):
                print(f"At file {i}of{len(pdf_names)}")                
                chunks = chunk_text_with_overlap(txt_file=pdf_names[i][1])
                print(len(chunks))
                sum = summarizer.summarize(chunks,max_token=100)
                results.append((pdf_names[i][0], sum))
                record = {
                        "file_name": pdf_names[i][0],
                        "summary": sum 
                        }
                f.write(json.dumps(record) + "\n")
#For pdf[0] after 14m on M1pro:
#<----Combined--->    The text discusses effective data incident management in healthcare, emphasizing email notification and adherence to organization-specific procedures. It also highlights understanding the causal chain of data incidents, identifying threat actors and vulnerabilities, and managing existing risks. Organizations are advised to strengthen their data incident management by implementing security controls and sharing best practices. Vulnerabilities are shared with trusted entities. The European Union's funding for humanitarian aid is mentioned, with the Centre for Humanitarian Data undertaking humanitarian activities with key partners.


25
('0e21835a42a6df2405496f62647058ff855743c1_normal.txt', PosixPath('Data/PDFs/0e21835a42a6df2405496f62647058ff855743c1_normal.txt'))
('1dcf57a5007b56254583423ba31107d22459bccf_normal.txt', PosixPath('Data/PDFs/1dcf57a5007b56254583423ba31107d22459bccf_normal.txt'))
('2009.08453v2_normal.txt', PosixPath('Data/PDFs/2009.08453v2_normal.txt'))
('2010.05981v2_normal.txt', PosixPath('Data/PDFs/2010.05981v2_normal.txt'))
('2010.11929v2_normal.txt', PosixPath('Data/PDFs/2010.11929v2_normal.txt'))
('2544CYX3TC3T5QB2NTVXD3IUFM654GXK_normal.txt', PosixPath('Data/PDFs/2544CYX3TC3T5QB2NTVXD3IUFM654GXK_normal.txt'))
('281928eff64137efdd144a833c81ad0ee45284c1_normal.txt', PosixPath('Data/PDFs/281928eff64137efdd144a833c81ad0ee45284c1_normal.txt'))
('2FDPTMT2NZDE6RIJSZZXGBMD7LYL7YHV_ocr.txt', PosixPath('Data/PDFs/2FDPTMT2NZDE6RIJSZZXGBMD7LYL7YHV_ocr.txt'))
('2G54QACZZK5MIIKK25USTLNPN66FST63_normal.txt', PosixPath('Data/PDFs/2G54QACZZK5MIIKK25USTLNPN66FST63_normal.txt'))
('2YS3ALM6OTD5ENWN4Z5LOBWG73575

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

<----Combined---> A data incident in humanitarian response refers to an event where data management causes harm or potential harm to crisis-affected populations, organizations, or individuals. These events exploit or exacerbate existing vulnerabilities and may create new ones. (Source: Centre for Humanitarian Data, August 2019) Let me know if you need any further assistance! Humanitarians face challenges in preventing and managing data incidents, which can compromise sensitive information and create new vulnerabilities. There is currently a lack of standardization and a common understanding of what constitutes a data incident. Developing tools and procedures for data incident management will be crucial for the humanitarian sector to address these challenges and ensure the protection of sensitive information. This will require collaboration and investment in technology and training. Effective management of data incidents will also help to prevent future data breaches and protect human r

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

<----Combined---> Data incidents in humanitarian response refer to events where data management causes harm or potential harm to crisis-affected populations, organizations, or individuals. These events exploit existing vulnerabilities and create new ones. Humanitarians face challenges in preventing and managing data incidents, which can compromise sensitive information and create new vulnerabilities. Standardization and a common understanding of data incidents are lacking, making it difficult to address these challenges. Developing tools and procedures for data incident management is crucial to protect sensitive information and ensure the sector's safety. Effective data management is essential to ensure that information is accurate, reliable, and accessible. Data responsibility involves ensuring that data is handled with care, that data is accessible to those who need it, and that data is used in a way that respects the rights and dignity of individuals and communities. Collaboration a

...

<----Combined---> Researchers explored self-supervised learning methods to improve the Vision Transformer (ViT) model's performance in image classification. A new optimization method, Adam, was proposed, and pre-training on smaller datasets and scaling up the model depth to 64 layers improved results. This study highlights the importance of data quality and model design. Additionally, a study on mindfulness found that Mindfulness-Based Stress Reduction (MBSR) may be a useful adjunct to traditional treatments for depression. Researchers aim to investigate the effectiveness of To examine the effectiveness of Mindfulness-Based Stress Reduction (MBSR) in reducing symptoms of Generalized Anxiety Disorder (GAD) and improving quality of life. The results suggest that MBSR is a useful treatment option for individuals with GAD, and can lead to significant reductions in anxiety symptoms and improvements in quality of life. The treatment can be adapted to different settings and populations, and c

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<----Combined---> Researchers used self-supervised learning to improve the Vision Transformer (ViT) model's performance in image classification. A new optimization method, Adam, was proposed and pre-training on smaller datasets and scaling up the model depth to 64 layers improved results. The study highlights the importance of data quality and model design. Additionally, a study on mindfulness found that Mindfulness-Based Stress Reduction (MBSR) may be a useful adjunct to traditional treatments for depression. Researchers are investigating the effectiveness of MBSR *   MBSR can help individuals with GAD reduce anxiety symptoms and improve quality of life.*   The treatment can be adapted to different settings and populations.*   MBSR can be a valuable adjunct to other treatments.
At file 5of25
7896


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

KeyboardInterrupt: 